In [ ]:
using System.Net;
using System.Net.Http;
using System.Net.NetworkInformation;
using System.Text.Json;

In [ ]:
public class BinanceApp
{
    private const string BASE_URL = "https://api.binance.com";

    private bool isRunning = true;

    private HttpClient httpClient = null;

    private BinanceExchangeService exchangeService = null;
    private BinancePingService pingService = null;
    private BinanceTimeService timeService = null;

    public BinanceApp()
    {
        App.HttpClient = new HttpClient();

        this.exchangeService = new BinanceExchangeService(App.HttpClient, BASE_URL, "/api/v3/exchangeInfo?symbol={0}");
        this.pingService = new BinancePingService(App.HttpClient, BASE_URL, "/api/v3/ping");
        this.timeService = new BinanceTimeService(App.HttpClient, BASE_URL, "/api/v3/time");
    }

    public void StartApp()
    {
        if (!this.TryPingBaseUrl() || !this.TryApiPingRequest()) {
            Console.WriteLine("Failed to ping app.");
            return;
        }

        this.isRunning = true;
    }

    public void UpdateApp()
    {
        if (!this.TryGetServerTime()) {
            Console.WriteLine("Failed to get server time.");
            return;
        }

        if (!this.TryGetAssetInfo("BTCUSDT")) {
            Console.WriteLine("Failed to get asset info.");
            return;
        }
    }

    private bool TryGetAssetInfo(string pairName)
    {
        Task<BinanceAsset> exchangeAssetTask = Task.Run<BinanceAsset>(async () => await this.exchangeService.GetAsset(pairName));

        if (exchangeAssetTask.Result != null) {
            // Console.WriteLine("Server time: " + serverTimeTask.Result.ToString());
            return true;
        }

        return false;
    }

    /// Gets the server time.
    private bool TryGetServerTime()
    {
        Task<DateTime> serverTimeTask = Task.Run<DateTime>(async () => await this.timeService.GetServerTime());

        if (serverTimeTask.Result != null) {
            // Console.WriteLine("Server time: " + serverTimeTask.Result.ToString());
            return true;
        }

        return false;
    }

    /// Ping ap ping path to check if the app is running.
    private bool TryApiPingRequest()
    {
        Task<bool> updateTask = Task.Run<bool>(async () => await this.pingService.TryApiPingRequest());
        return updateTask.Result;
    }

    /// Ping the base url to check if the app is running.
    private bool TryPingBaseUrl()
    {
        Task<bool> updateTask = Task.Run<bool>(async () => await this.pingService.TryPingBaseUrl());
        return updateTask.Result;
    }
}

public class BinanceAsset
{
    public string assetName { get; set; }
    public string timeFrame { get; set; }
    public double price { get; set; }
}

public class BinanceExchangeService
{
    private HttpClient httpClient = null;
    private string apiBaseUrl = null;
    private string assetPath = null;

    public BinanceExchangeService(HttpClient httpClient, string baseUrl, string assetPath)
    {
        App.HttpClient = httpClient;
        App.BaseUrl = baseUrl;
        this.assetPath = assetPath;
    }

    public async Task<BinanceAsset> GetAsset(string pairName)
    {
        try {
            string endPoint = string.Format($"{App.BaseUrl}{this.assetPath}", pairName);

            HttpResponseMessage apiAssetResult = await App.HttpClient.GetAsync(endPoint);

            HttpResponseMessageHandler.LogResponse((apiAssetResult));

            if (apiAssetResult.StatusCode == HttpStatusCode.OK) {
                string apiAssetResultContent = await apiAssetResult.Content.ReadAsStringAsync();     
                Dictionary<string, object> values = JsonSerializer.Deserialize<Dictionary<string, object>>(apiAssetResultContent);

                Console.WriteLine($"Asset: {apiAssetResultContent}");

                // BinanceAsset assets = JsonSerializer.Deserialize<BinanceAsset>(responseString);

                return null;
            }

            return null;
        }
        catch (Exception e) {
            ExceptionHandler.LogException(e);
        }

        return null;
    }
}

public class BinancePingService
{
    private const string API_PING_METHOD = "GET";

    private string apiBaseUrl = string.Empty;
    private string apiPingPath = string.Empty;

    private HttpClient httpClient = null;

    public BinancePingService(HttpClient httpClient, string apiBaseUrl, string apiPingPath)
    {
        App.HttpClient = httpClient;
        App.BaseUrl = apiBaseUrl;
        this.apiPingPath = apiPingPath;
    }

    public async Task<bool> TryApiPingRequest()
    {
        try {
            string endPoint = $"{App.BaseUrl}{this.apiPingPath}";
            HttpResponseMessage apiPingresult = await App.HttpClient.GetAsync(endPoint);

            // HttpResponseMessageHandler.LogResponse((apiPingresult));

            return apiPingresult.StatusCode == HttpStatusCode.OK;
        }
        catch (Exception e) {
            ExceptionHandler.LogException(e);
        }

        return false;
    }

    public async Task<bool> TryPingBaseUrl()
    {
        try {
            Ping ping = new Ping();
            // Ping url must not contain {https://}.
            PingReply pingResult = await ping.SendPingAsync(App.BaseUrl.Replace("https://", ""));

            // PingReplyHandler.LogReply(pingResult);
            
            return pingResult.Status == IPStatus.Success;
        } catch (Exception e) {
            ExceptionHandler.LogException(e);
        }

        return false;
    }
}

public class BinanceTimeService
{
    private const string API_TIME_METHOD = "GET";

    private string apiBaseUrl = string.Empty;
    private string apiTimePath = string.Empty;

    private DateTime invalidTime = new DateTime(1970, 1, 1, 0, 0, 0, DateTimeKind.Utc);

    private HttpClient httpClient = null;

    public BinanceTimeService(HttpClient httpClient, string apiBaseUrl, string apiTimePath)
    {
        App.HttpClient = httpClient;
        App.BaseUrl = apiBaseUrl;
        this.apiTimePath = apiTimePath;
    }

    public async Task<DateTime> GetServerTime()
    {
        try {
            string endPoint = $"{App.BaseUrl}{this.apiTimePath}";
            HttpResponseMessage apiTimeResult = await App.HttpClient.GetAsync(endPoint);

            // HttpResponseMessageHandler.LogResponse((apiTimeResult));

            if (apiTimeResult.StatusCode == HttpStatusCode.OK) {
                string apiTimeResultContent = await apiTimeResult.Content.ReadAsStringAsync();
                Dictionary<string, object> values = JsonSerializer.Deserialize<Dictionary<string, object>>(apiTimeResultContent);

                if (Double.TryParse(values["serverTime"].ToString(), out double asDouble)) {
                    DateTime start = new DateTime(1970, 1, 1, 0, 0, 0, DateTimeKind.Utc);
                    DateTime serverTime = start.AddMilliseconds((long)asDouble);

                    Console.WriteLine("Server time: " + serverTime.ToString());

                    return serverTime;
                }
            }

            return this.invalidTime;
        }
        catch (Exception e) {
            ExceptionHandler.LogException(e);
        }

        return this.invalidTime;
    }
}

public static class ExceptionHandler
{
    private static StringBuilder messageBuilder = new StringBuilder();

    public static void LogException(Exception e)
    {
        messageBuilder.AppendLine($"___MESSAGE:{e.Message}.");
        messageBuilder.AppendLine($"___STACKTRACE:{e.StackTrace}.");
        
        Console.WriteLine(messageBuilder.ToString());
    }
}

public static class HttpResponseMessageHandler
{
    private static StringBuilder messageBuilder = new StringBuilder();

    public static async void LogResponse(HttpResponseMessage response)
    {
        string result = await response.Content.ReadAsStringAsync();

        messageBuilder.AppendLine($"___STATUS:{response.StatusCode}.");
        messageBuilder.AppendLine($"___CONTENT:{result}.");

        Console.WriteLine(messageBuilder.ToString());
    }
}

public static class PingReplyHandler
{
    private static StringBuilder messageBuilder = new StringBuilder();

    public static void LogReply(PingReply reply)
    {
        messageBuilder.AppendLine($"__ADDRESS: {reply.Address}");
        messageBuilder.AppendLine($"__ROUNTRIP TIME: {reply.RoundtripTime}");
        messageBuilder.AppendLine($"__BUFFER: {reply.Buffer.Length}");
        messageBuilder.AppendLine($"__STATUS: {reply.Status}");

        Console.WriteLine(messageBuilder.ToString());
    }
}

BinanceApp app = new BinanceApp();
app.StartApp();
app.UpdateApp();

Server time: 5/19/2022 10:28:19 PM
___STATUS:OK.
___CONTENT:{"timezone":"UTC","serverTime":1652999300172,"rateLimits":[{"rateLimitType":"REQUEST_WEIGHT","interval":"MINUTE","intervalNum":1,"limit":1200},{"rateLimitType":"ORDERS","interval":"SECOND","intervalNum":10,"limit":50},{"rateLimitType":"ORDERS","interval":"DAY","intervalNum":1,"limit":160000},{"rateLimitType":"RAW_REQUESTS","interval":"MINUTE","intervalNum":5,"limit":6100}],"exchangeFilters":[],"symbols":[{"symbol":"BTCUSDT","status":"TRADING","baseAsset":"BTC","baseAssetPrecision":8,"quoteAsset":"USDT","quotePrecision":8,"quoteAssetPrecision":8,"baseCommissionPrecision":8,"quoteCommissionPrecision":8,"orderTypes":["LIMIT","LIMIT_MAKER","MARKET","STOP_LOSS_LIMIT","TAKE_PROFIT_LIMIT"],"icebergAllowed":true,"ocoAllowed":true,"quoteOrderQtyMarketAllowed":true,"allowTrailingStop":false,"isSpotTradingAllowed":true,"isMarginTradingAllowed":true,"filters":[{"filterType":"PRICE_FILTER","minPrice":"0.01000000","maxPrice":"1000000.000000